# 1. Load the Preprocessed Data

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
import gc

# Indicate if the script is running on Google Colab or not
using_colab = 'google.colab' in sys.modules

if using_colab:
    # Connect Google Drive to Colab
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    
    # Install necessary packages
    !pip install numpy
    !pip install pandas
    !pip install sklearn
    !pip install matplotlib
    !pip install seaborn
    !pip install imbalanced-learn
    !pip install shap
    
    # Path for Google Colab
    project_root = '/content/gdrive/MyDrive/oc_projet_4/' 
else:
    # Get the current working directory as base directory for the notebook
    base_dir = os.getcwd()
    
    # Adjust the project root path relatively to where the notebook is located
    # Assuming the notebook is inside a 'notebooks' directory and we need to go up one level to access project root
    project_root = os.path.join(base_dir, '..')

# Clean output of cell
from IPython.display import clear_output
clear_output()

# Load the CSV files
train_domain_path = os.path.join(project_root, 'data', 'app_train_domain.csv')
test_domain_path = os.path.join(project_root, 'data', 'app_test_domain.csv')

app_train_domain = pd.read_csv(train_domain_path) # Load the training data
app_test_domain = pd.read_csv(test_domain_path) # Load the test data

# Extract the training and testing sets
X_train = app_train_domain.drop(columns=['TARGET']).values # Features for training
y_train = app_train_domain['TARGET'].values # Target variable for training
X_test = app_test_domain.values  # Assuming test set does not have TARGET column

# Split the training data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print('Data loaded successfully.')
print(f'X_train shape: {X_train.shape}') # X_train is used for training the model
print(f'X_val shape: {X_val_split.shape}') # X_val is used for validation during training
print(f'X_test shape: {X_test.shape}') # X_test is the test set for which predictions are made
print(f'y_train shape: {y_train.shape}') # y_train is the target variable for training
print(f'y_val shape: {y_val_split.shape}') # y_val is the target variable for validation


Data loaded successfully.
X_train shape: (307511, 247)
X_val shape: (61503, 247)
X_test shape: (48744, 248)
y_train shape: (307511,)
y_val shape: (61503,)


In [3]:
# Check the first few values of y_train
print('First 10 values of y_train:', y_train[:10])

# Check the unique values in y_train
print('Unique values in y_train:', np.unique(y_train))

# Check for missing values in y_train
print('Number of missing values in y_train:', pd.Series(y_train).isnull().sum())

First 10 values of y_train: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Unique values in y_train: [0. 1.]
Number of missing values in y_train: 0


In [4]:
# Display the first few rows of the training data
print(f"y_train shape: {y_train.shape}")
print(f"y_train sample values: {y_train[:5]}")  # Afficher les premières valeurs pour vérification

# Display the columns of X_train to verify the features
x_train_columns = list(app_train_domain.drop(columns=['TARGET']).columns)
print(f"X_train columns: {x_train_columns}")

# List of new features to check
new_features = ['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']

# Check if these new features are in X_train columns
missing_features = [feature for feature in new_features if feature not in x_train_columns]

if not missing_features:
    print("All new features are present in X_train.")
else:
    print(f"Missing features in X_train: {missing_features}")


y_train shape: (307511,)
y_train sample values: [1. 0. 0. 0. 0.]
X_train columns: ['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AV

In [5]:
# Verify the data types and missing values in the training data
print(app_train_domain.dtypes)
print(app_train_domain.isnull().sum())


SK_ID_CURR                  int64
TARGET                    float64
NAME_CONTRACT_TYPE          int64
FLAG_OWN_CAR                int64
FLAG_OWN_REALTY             int64
                           ...   
DAYS_EMPLOYED_ANOM           bool
CREDIT_INCOME_PERCENT     float64
ANNUITY_INCOME_PERCENT    float64
CREDIT_TERM               float64
DAYS_EMPLOYED_PERCENT     float64
Length: 248, dtype: object
SK_ID_CURR                0
TARGET                    0
NAME_CONTRACT_TYPE        0
FLAG_OWN_CAR              0
FLAG_OWN_REALTY           0
                         ..
DAYS_EMPLOYED_ANOM        0
CREDIT_INCOME_PERCENT     0
ANNUITY_INCOME_PERCENT    0
CREDIT_TERM               0
DAYS_EMPLOYED_PERCENT     0
Length: 248, dtype: int64


In [6]:
import os
import pandas as pd
import numpy as np
from scipy.stats import shapiro, zscore

using_colab = 'google.colab' in sys.modules

if using_colab:
    # Load the CSV files using relative paths
    train_domain_path = os.path.join(project_root, 'data', 'app_train_domain.csv')
    test_domain_path = os.path.join(project_root, 'data', 'app_test_domain.csv')
else:
# Get the current working directory as base directory for the notebook
    base_dir = os.getcwd()
    
    # Adjust the project root path relatively to where the notebook is located
    # Assuming the notebook is inside a 'notebooks' directory and we need to go up one level to access project root
    project_root = os.path.join(base_dir, '..')

# Load the CSV files using relative paths
train_domain_path = os.path.join(project_root, 'data', 'app_train_domain.csv')
test_domain_path = os.path.join(project_root, 'data', 'app_test_domain.csv')

train_data = pd.read_csv(train_domain_path)  # Load the training data
test_data = pd.read_csv(test_domain_path)    # Load the test data

# Features to analyze
features_to_analyze = [
    'CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM',
    'DAYS_EMPLOYED_PERCENT', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'
]

# Shapiro-Wilk Test for normality
def test_normality(df, features):
    normality_results = {}
    for feature in features:
        stat, p = shapiro(df[feature].dropna())  # Ignore missing values for the test
        normality_results[feature] = {'Statistic': stat, 'p-value': p, 'Normal': p > 0.05}
    return normality_results

# Detecting outliers using z-score
def detect_outliers(df, features):
    outliers_results = {}
    for feature in features:
        z_scores = zscore(df[feature].dropna())  # Ignore missing values for z-score calculation
        outliers = np.where(np.abs(z_scores) > 3)[0]  # Outliers with z-score > 3
        outliers_results[feature] = df[feature].iloc[outliers]
    return outliers_results

# Execute the functions
normality_results = test_normality(train_data, features_to_analyze)
outliers_results = detect_outliers(train_data, features_to_analyze)

# Display the results
print("Normality Test Results (Shapiro-Wilk):")
for feature, result in normality_results.items():
    print(f"{feature}: Statistic={result['Statistic']}, p-value={result['p-value']}, Normal={result['Normal']}")

print("\nDetected Outliers:")
for feature, outliers in outliers_results.items():
    print(f"{feature}: {len(outliers)} outliers")
    print(outliers)


scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 307511.


Normality Test Results (Shapiro-Wilk):
CREDIT_INCOME_PERCENT: Statistic=0.8683387382469063, p-value=7.017665490743911e-144, Normal=False
ANNUITY_INCOME_PERCENT: Statistic=0.9120356975923316, p-value=1.7391184523648002e-131, Normal=False
CREDIT_TERM: Statistic=0.8813492261004188, p-value=1.2529197766384718e-140, Normal=False
DAYS_EMPLOYED_PERCENT: Statistic=0.8455920689876675, p-value=6.2034854179349085e-149, Normal=False
EXT_SOURCE_1: Statistic=0.8413734001601811, p-value=8.494600632668699e-150, Normal=False
EXT_SOURCE_2: Statistic=0.9267468129817444, p-value=4.763031590479468e-126, Normal=False
EXT_SOURCE_3: Statistic=0.9644152629779538, p-value=9.85564431307928e-106, Normal=False
DAYS_BIRTH: Statistic=0.9696564518865638, p-value=1.709357664268864e-101, Normal=False

Detected Outliers:
CREDIT_INCOME_PERCENT: 4340 outliers
176       12.835714
270       13.754462
371       13.611000
380       12.112000
678       34.916667
            ...    
307369    13.742400
307401    12.729433
30744

### Analysis and Interpretation

#### Normality Test Results (Shapiro-Wilk)

The Shapiro-Wilk test was applied to each feature to assess normality. Here are the results:

- **CREDIT_INCOME_PERCENT**: Statistic=0.868, p-value=7.018e-144, Normal=False
- **ANNUITY_INCOME_PERCENT**: Statistic=0.912, p-value=1.739e-131, Normal=False
- **CREDIT_TERM**: Statistic=0.881, p-value=1.253e-140, Normal=False
- **DAYS_EMPLOYED_PERCENT**: Statistic=0.846, p-value=6.203e-149, Normal=False
- **EXT_SOURCE_1**: Statistic=0.841, p-value=8.495e-150, Normal=False
- **EXT_SOURCE_2**: Statistic=0.927, p-value=4.763e-126, Normal=False
- **EXT_SOURCE_3**: Statistic=0.964, p-value=9.856e-106, Normal=False
- **DAYS_BIRTH**: Statistic=0.970, p-value=1.709e-101, Normal=False

**Interpretation**:
- All the p-values are significantly smaller than 0.05, which means that none of the features follow a normal distribution. This suggests that standardization (which assumes normality) may not be the best preprocessing step. Instead, other scaling methods like Min-Max Scaling or Robust Scaler may be more appropriate.

#### Detected Outliers

The z-score method was used to detect outliers in each feature. Here are the results:

- **CREDIT_INCOME_PERCENT**: 4340 outliers
- **ANNUITY_INCOME_PERCENT**: 3792 outliers
- **CREDIT_TERM**: 106 outliers
- **DAYS_EMPLOYED_PERCENT**: 6975 outliers
- **EXT_SOURCE_1**: 1593 outliers
- **EXT_SOURCE_2**: 0 outliers
- **EXT_SOURCE_3**: 0 outliers
- **DAYS_BIRTH**: 0 outliers

**Interpretation**:
- **CREDIT_INCOME_PERCENT**, **ANNUITY_INCOME_PERCENT**, **DAYS_EMPLOYED_PERCENT**, and **EXT_SOURCE_1** have a significant number of outliers. This indicates that these features have extreme values which could potentially skew the results of the analysis or modeling.
- **EXT_SOURCE_2**, **EXT_SOURCE_3**, and **DAYS_BIRTH** do not have any outliers detected with the z-score method, suggesting these features are more stable and less prone to extreme values.

### Recommendations

Given these findings, here are some recommendations for preprocessing and modeling:

1. **Normalization/Scaling**:
   - Since the features do not follow a normal distribution, consider using **Min-Max Scaling** or **Robust Scaler** instead of standardization. Robust Scaler is particularly useful for features with many outliers as it scales the data using statistics that are robust to outliers.

2. **Handling Outliers**:
   - For features with significant outliers, consider applying techniques such as:
     - **Winsorizing**: Capping the extreme values.
     - **Transformation**: Applying log, square root, or other transformations to reduce the effect of outliers.
     - **Removal**: Removing data points that are identified as outliers, although this should be done cautiously to avoid losing valuable information.

3. **Feature Engineering**:
   - Investigate if some of these outliers are due to data entry errors or other anomalies that can be corrected.
   - For features with many outliers, you can create additional features that capture the presence of outliers, e.g., a binary feature indicating if a value is an outlier or not.

4. **Modeling**:
   - Use models that are less sensitive to outliers, such as tree-based methods (e.g., Random Forest, Gradient Boosting) which are generally more robust to outliers.
   - Ensure that you have robust cross-validation practices to evaluate the impact of these preprocessing steps.

### Next Steps

- Implement Min-Max Scaling or Robust Scaling on the features.
- Apply outlier handling techniques as needed.
- Re-evaluate the performance of your model with these preprocessing steps.
- Ensure that your evaluation metrics align with the business goals, such as minimizing the cost of prediction errors as outlined in your project requirements.

By following these recommendations, you can ensure that your data preprocessing is robust and your models are well-prepared to handle the characteristics of your dataset.

# 2.Hyperparameter Tuning via GridSearchCV and Imbalanced-learn Pipeline

The objective of this code is to:
1. **Set Up Resampling Techniques**: Define over-sampling (SMOTE) and under-sampling (RandomUnderSampler) methods to handle class imbalance in the dataset.
2. **Initialize Models**: Create instances of RandomForestClassifier and RidgeClassifier for classification tasks.
3. **Define Hyperparameter Grids**: Specify parameter grids for tuning hyperparameters of the classifiers using GridSearchCV.
4. **Create Pipelines**: Construct pipelines to integrate resampling techniques with the classifiers.
5. **Perform Hyperparameter Tuning**: Use GridSearchCV to find the best hyperparameters and resampling techniques for each classifier, evaluating them using cross-validation and storing the best models.

In [7]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import gc

# Define the resampling techniques
over_sampler = SMOTE(random_state=42)  # Over-sampling method
under_sampler = RandomUnderSampler(random_state=42)  # Under-sampling method

# Define the models
rf = RandomForestClassifier(random_state=42)  # Random Forest Classifier
ridge = RidgeClassifier()  # Create an instance of the Ridge Classifier

# Define the parameter grids for GridSearchCV
param_grid_rf = {  # Parameter grid for Random Forest
    'classifier__n_estimators': [100, 200],  # Number of trees in the forest
    'classifier__max_depth': [10, 20]  # Maximum depth of the tree
}

param_grid_ridge = {  # Parameter grid for Ridge Classifier
    'classifier__alpha': [1.0, 0.1, 0.01]  # Regularization strength
}

# Define pipelines with Robust Scaling
pipeline_rf_over = Pipeline([  # Pipeline for Random Forest with over-sampling
    ('scaler', RobustScaler()),  # Robust Scaling
    ('oversample', over_sampler),  # Over-sampling method
    ('classifier', rf)  # Random Forest Classifier
])

pipeline_rf_under = Pipeline([  # Pipeline for Random Forest with under-sampling
    ('scaler', RobustScaler()),  # Robust Scaling
    ('undersample', under_sampler),  # Under-sampling method
    ('classifier', rf)  # Random Forest Classifier
])

pipeline_rf_weight = Pipeline([  # Pipeline for Random Forest with class weight adjustment
    ('scaler', RobustScaler()),  # Robust Scaling
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))  # Random Forest Classifier with class weight adjustment
])

pipeline_ridge_over = Pipeline([  # Pipeline for Ridge Classifier with over-sampling
    ('scaler', RobustScaler()),  # Robust Scaling
    ('oversample', over_sampler),  # Over-sampling method
    ('classifier', ridge)  # Ridge Classifier
])

pipeline_ridge_under = Pipeline([  # Pipeline for Ridge Classifier with under-sampling
    ('scaler', RobustScaler()),  # Robust Scaling
    ('undersample', under_sampler),  # Under-sampling method
    ('classifier', ridge)  # Ridge Classifier
])

pipeline_ridge_weight = Pipeline([  # Pipeline for Ridge Classifier with class weight adjustment
    ('scaler', RobustScaler()),  # Robust Scaling
    ('classifier', RidgeClassifier(class_weight='balanced'))  # Ridge Classifier with class weight adjustment
])

# Function to compute ROC AUC for models with or without predict_proba
def compute_roc_auc(model, X, y):
    if hasattr(model, "predict_proba"): # Check if the model has predict_proba attribute
        y_proba = model.predict_proba(X)[:, 1] # Get the probability of the positive class
    elif hasattr(model, "decision_function"): # Check if the model has decision_function attribute
        y_proba = model.decision_function(X) # Get the decision function scores
    else:
        y_proba = model.predict(X) # Use the model's predictions as the scores
    return roc_auc_score(y, y_proba) # Compute ROC AUC score

# Perform GridSearchCV for each pipeline and store the best estimators
grids = [
    (pipeline_rf_over, param_grid_rf, 'Random Forest with Over-sampling'), # pipeline, parameter grid, name
    (pipeline_rf_under, param_grid_rf, 'Random Forest with Under-sampling'),
    (pipeline_rf_weight, param_grid_rf, 'Random Forest with Class Weight'),
    (pipeline_ridge_over, param_grid_ridge, 'Ridge with Over-sampling'),
    (pipeline_ridge_under, param_grid_ridge, 'Ridge with Under-sampling'),
    (pipeline_ridge_weight, param_grid_ridge, 'Ridge with Class Weight')
]

best_estimators = {}  # Dictionary to store the best estimators
roc_auc_scores = {}  # Dictionary to store the ROC AUC scores for each model

for pipeline, param_grid, name in grids:  # Iterate over the pipelines
    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='roc_auc', cv=3, n_jobs=-1)  # GridSearchCV for hyperparameter tuning
    grid_search.fit(X_train_split, y_train_split)  # Fit the model for training data
    best_estimators[name] = grid_search.best_estimator_  # Store the best estimator
    roc_auc = compute_roc_auc(grid_search.best_estimator_, X_val_split, y_val_split)  # Compute ROC AUC on validation set
    roc_auc_scores[name] = roc_auc  # Store the ROC AUC score
    print(f"Best parameters for {name}: {grid_search.best_params_}")  # Print the best parameters
    print(f"Best ROC AUC for {name}: {roc_auc}")  # Print the best ROC AUC score

# Select the best model based on ROC AUC scores
best_model_name = max(roc_auc_scores, key=roc_auc_scores.get) #key is a function that defines the sorting criteria
best_model = best_estimators[best_model_name] # Get the best model
print(f"\nBest model: {best_model_name} with ROC AUC: {roc_auc_scores[best_model_name]}")

gc.collect()  # Garbage collection to free up memory. Works only in Jupyter notebooks.


Best parameters for Random Forest with Over-sampling: {'classifier__max_depth': 20, 'classifier__n_estimators': 200}
Best ROC AUC for Random Forest with Over-sampling: 0.7051265054419742
Best parameters for Random Forest with Under-sampling: {'classifier__max_depth': 20, 'classifier__n_estimators': 200}
Best ROC AUC for Random Forest with Under-sampling: 0.7439398146413645
Best parameters for Random Forest with Class Weight: {'classifier__max_depth': 10, 'classifier__n_estimators': 200}
Best ROC AUC for Random Forest with Class Weight: 0.7334545861438759
Best parameters for Ridge with Over-sampling: {'classifier__alpha': 1.0}
Best ROC AUC for Ridge with Over-sampling: 0.7451146797593615
Best parameters for Ridge with Under-sampling: {'classifier__alpha': 1.0}
Best ROC AUC for Ridge with Under-sampling: 0.7483156966485889
Best parameters for Ridge with Class Weight: {'classifier__alpha': 1.0}
Best ROC AUC for Ridge with Class Weight: 0.7493240652562565

Best model: Ridge with Class Weig

680

### Interpretation of the Results

#### Summary

1. **Random Forest with Over-sampling**:
   - **Best Parameters**: `{'classifier__max_depth': 20, 'classifier__n_estimators': 200}`
   - **Best ROC AUC**: `0.7025714300577274`

2. **Random Forest with Under-sampling**:
   - **Best Parameters**: `{'classifier__max_depth': 20, 'classifier__n_estimators': 200}`
   - **Best ROC AUC**: `0.7393607794366517`

3. **Random Forest with Class Weight**:
   - **Best Parameters**: `{'classifier__max_depth': 10, 'classifier__n_estimators': 200}`
   - **Best ROC AUC**: `0.7284622555457663`

4. **Ridge with Over-sampling**:
   - **Best Parameters**: `{'classifier__alpha': 1.0}`
   - **Best ROC AUC**: `0.7413334928962128`

5. **Ridge with Under-sampling**:
   - **Best Parameters**: `{'classifier__alpha': 1.0}`
   - **Best ROC AUC**: `0.7449914888258974`

6. **Ridge with Class Weight**:
   - **Best Parameters**: `{'classifier__alpha': 1.0}`
   - **Best ROC AUC**: `0.7468065211237337`

#### Observations

1. **Random Forest**:
   - The best parameters for both over-sampling and under-sampling are the same, suggesting that the model configuration is robust across different resampling strategies.
   - The ROC AUC for under-sampling (0.7394) is higher than for over-sampling (0.7026), indicating that under-sampling might be a better strategy for the Random Forest model in this case.
   - Using class weight adjustment in Random Forest shows an improvement in ROC AUC compared to over-sampling but still underperforms compared to under-sampling.

2. **Ridge Classifier**:
   - The best parameters for both over-sampling and under-sampling are the same.
   - The ROC AUC values are very close for over-sampling (0.7413) and under-sampling (0.7450), indicating that both resampling strategies perform similarly well for the Ridge Classifier.
   - Using class weight adjustment in Ridge Classifier gives a slightly better ROC AUC (0.7468), making it the best-performing method overall.

### Conclusion

- **Random Forest** performs best with under-sampling, suggesting that reducing the number of majority class samples can help the model learn better.
- **Ridge Classifier** performs best with class weight adjustment, indicating that this method handles class imbalance effectively.
- Overall, class weight adjustment in the Ridge Classifier achieved the highest ROC AUC, making it the most effective method for this dataset.

# 3.Evaluate the Best Model

The objective of this code is to:
1. **Select the Best Model**: Identify and select the best model from the hyperparameter-tuned models stored in `best_estimators` based on their performance on the test set.
2. **Make Predictions**: Use the selected best model to predict the target variable on the test dataset.
3. **Evaluate Performance**: Calculate the confusion matrix and ROC AUC score to assess the model's performance on the test data.
4. **Check for Overfitting**: Ensure that the ROC AUC score is below 0.82 to avoid overfitting, issuing a warning if the score is higher.
5. **Generate Detailed Metrics**: Print a comprehensive classification report including precision, recall, and F1-score for a detailed evaluation of the model's performance.

In [8]:
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report

# Function to compute ROC AUC for models with or without predict_proba
def compute_roc_auc(model, X, y): # Compute ROC AUC for models with or without predict_proba
    if hasattr(model, "predict_proba"): # Check if the model has predict_proba attribute; predict_proba is used for models like RandomForestClassifier
        y_proba = model.predict_proba(X)[:, 1] # Get the probability of the positive class
    elif hasattr(model, "decision_function"): # Check if the model has decision_function attribute; decision_function is used for models like SVC
        y_proba = model.decision_function(X) # Get the decision function scores
    else: 
        y_proba = model.predict(X) # Use the model's predictions as the scores
    return roc_auc_score(y, y_proba) # Compute ROC AUC score

# Select the best model based on validation ROC AUC score
best_model_name = max(best_estimators, key=lambda name: compute_roc_auc(best_estimators[name], X_val_split, y_val_split)) 
best_model = best_estimators[best_model_name] # Get the best model

# Predict the target variable on the validation set
y_val_pred = best_model.predict(X_val_split) # Predict the target variable on the validation set
y_val_proba = best_model.predict_proba(X_val_split)[:, 1] if hasattr(best_model, "predict_proba") else best_model.decision_function(X_val_split) if hasattr(best_model, "decision_function") else best_model.predict(X_val_split) # Get the predicted probabilities

# Compute confusion matrix for validation set
conf_matrix_val = confusion_matrix(y_val_split, y_val_pred)
print("Confusion Matrix (Validation):")
print(conf_matrix_val)

# Compute ROC AUC score for validation set using probabilities
roc_auc_val = roc_auc_score(y_val_split, y_val_proba) 
print(f"ROC AUC (Validation): {roc_auc_val}")

# Ensure ROC AUC < 0.82 for validation set
if roc_auc_val >= 0.82: # Check if ROC AUC score is greater than or equal to 0.82 otherwise issue a warning; 0.82 is a threshold to avoid overfitting
    print("Warning: ROC AUC score (Validation) is greater than or equal to 0.82. Model might be overfitting.")

# Print classification report for validation set
print("Classification Report (Validation):")
print(classification_report(y_val_split, y_val_pred))



Confusion Matrix (Validation):
[[39011 17543]
 [ 1596  3353]]
ROC AUC (Validation): 0.7493240652562565
Classification Report (Validation):
              precision    recall  f1-score   support

         0.0       0.96      0.69      0.80     56554
         1.0       0.16      0.68      0.26      4949

    accuracy                           0.69     61503
   macro avg       0.56      0.68      0.53     61503
weighted avg       0.90      0.69      0.76     61503



### Interpretation of the Results

#### Confusion Matrix
```
Confusion Matrix (Validation):
[[39011 17543]
 [ 1596  3353]]
```
The confusion matrix shows:
- **True Negatives (TN)**: 39,011 (clients correctly identified as not defaulting)
- **False Positives (FP)**: 17,543 (clients incorrectly identified as defaulting)
- **False Negatives (FN)**: 1,596 (clients incorrectly identified as not defaulting)
- **True Positives (TP)**: 3,353 (clients correctly identified as defaulting)

#### ROC AUC Score
```
ROC AUC (Validation): 0.7493240652562565
```
The ROC AUC score is approximately 0.75, indicating a reasonable ability of the model to distinguish between defaulting and non-defaulting clients.

#### Classification Report
```
Classification Report (Validation):
              precision    recall  f1-score   support

         0.0       0.96      0.69      0.80     56554
         1.0       0.16      0.68      0.26      4949

    accuracy                           0.69     61503
   macro avg       0.56      0.68      0.53     61503
weighted avg       0.90      0.69      0.76     61503
```
- **Class 0 (Non-defaulting clients)**:
  - **Precision**: 0.96 (The proportion of clients predicted not to default that actually did not default)
  - **Recall**: 0.69 (The proportion of actual non-defaulting clients correctly predicted)
  - **F1-Score**: 0.80 (Harmonic mean of precision and recall)
  - **Support**: 56,554 (Number of actual non-defaulting clients)

- **Class 1 (Defaulting clients)**:
  - **Precision**: 0.16 (The proportion of clients predicted to default that actually defaulted)
  - **Recall**: 0.68 (The proportion of actual defaulting clients correctly predicted)
  - **F1-Score**: 0.26 (Harmonic mean of precision and recall, which is low due to precision being low)
  - **Support**: 4,949 (Number of actual defaulting clients)

- **Overall Metrics**:
  - **Accuracy**: 0.69 (The proportion of total correct predictions)
  - **Macro Average**: Average precision, recall, and F1-score for both classes (treating all classes equally)
  - **Weighted Average**: Average precision, recall, and F1-score for both classes (considering the support of each class)

### Observations and Insights
1. **Model Performance**:
   - The model shows a significant improvement in recall for defaulting clients (0.68), meaning it identified a large proportion of defaulting clients correctly.
   - However, the precision for defaulting clients is quite low (0.16), indicating many false positives.
   - The model performs well in predicting non-defaulting clients (Class 0) with high precision (0.96) and reasonable recall (0.69).

2. **ROC AUC Score**:
   - The ROC AUC score of 0.75 indicates that the model has a good ability to discriminate between defaulting and non-defaulting clients.

3. **Imbalance Issue**:
   - The model still faces an imbalance issue, as seen in the lower precision for defaulting clients and the disparity between class performances.

4. **Need for Improvement**:
   - Although the model has improved, there is still room for enhancement, particularly in reducing false positives for defaulting clients.

### Recommendations
1. **Reassess the Model**:
   - Consider fine-tuning the threshold for predicting defaulting clients to balance precision and recall better.
   - Explore ensemble methods like balanced random forests or XGBoost with tailored class weights to further address class imbalance.

2. **Feature Engineering**:
   - Further investigate and refine feature engineering to enhance model performance. Creating more relevant features can help the model differentiate better between classes.

3. **Evaluate Data**:
   - Ensure data quality and consider additional preprocessing steps, such as handling outliers and scaling features appropriately, to improve model performance.

4. **Post-Processing**:
   - Implement post-processing steps like cost-sensitive learning to account for the higher cost of false negatives, which could further improve decision-making.

By continuing to refine the handling of class imbalance, feature engineering, and model evaluation, the model's ability to predict both defaulting and non-defaulting clients can be significantly enhanced.

In [9]:
import lightgbm as lgb # Import the LightGBM model
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report # Import metrics for evaluation

# Initialize the LightGBM model with class weight to handle imbalance
lgbm_model = lgb.LGBMClassifier(random_state=42, class_weight='balanced') # LightGBM model with class weight adjustment

# Fit the model for validation set
lgbm_model.fit(X_train_split, y_train_split)

# Predict on the validation set
y_val_pred = lgbm_model.predict(X_val_split)

# Compute confusion matrix for validation set
conf_matrix_val = confusion_matrix(y_val_split, y_val_pred)
print("Confusion Matrix (Validation) with LightGBM:")
print(conf_matrix_val)

# Compute ROC AUC score for validation set
roc_auc_val = roc_auc_score(y_val_split, y_val_pred)
print(f"ROC AUC (Validation) with LightGBM: {roc_auc_val}")

# Print classification report for validation set
print("Classification Report (Validation) with LightGBM:")
print(classification_report(y_val_split, y_val_pred))


[LightGBM] [Info] Number of positive: 19876, number of negative: 226132
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062770 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12731
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 236
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Confusion Matrix (Validation) with LightGBM:
[[40247 16307]
 [ 1559  3390]]
ROC AUC (Validation) with LightGBM: 0.6983214911558947
Classification Report (Validation) with LightGBM:
              precision    recall  f1-score   support

         0.0       0.96      0.71      0.82     56554
         1.0       0.17      0.68      0.28      4949

    accuracy                           0.71     61503
   macro avg       0.57      0.70      0.

### Interpretation of the Results

#### Summary of Results

1. **Confusion Matrix**
```
Confusion Matrix (Validation) with LightGBM:
[[40247 16307]
 [ 1559  3390]]
```
- **True Negatives (TN)**: 40,247 (clients correctly identified as not defaulting)
- **False Positives (FP)**: 16,307 (clients incorrectly identified as defaulting)
- **False Negatives (FN)**: 1,559 (clients incorrectly identified as not defaulting)
- **True Positives (TP)**: 3,390 (clients correctly identified as defaulting)

2. **ROC AUC Score**
```
ROC AUC (Validation) with LightGBM: 0.6983214911558947
```
- The ROC AUC score of 0.698 indicates that the model has a reasonable discriminative ability between the classes (default vs. non-default).

3. **Classification Report**
```
Classification Report (Validation) with LightGBM:
              precision    recall  f1-score   support

         0.0       0.96      0.71      0.82     56554
         1.0       0.17      0.68      0.28      4949

    accuracy                           0.71     61503
   macro avg       0.57      0.70      0.55     61503
weighted avg       0.90      0.71      0.77     61503
```
- **Class 0 (Non-defaulting clients)**:
  - **Precision**: 0.96 (proportion of clients predicted not to default that actually did not default)
  - **Recall**: 0.71 (proportion of actual non-defaulting clients correctly predicted)
  - **F1-Score**: 0.82 (harmonic mean of precision and recall)
  - **Support**: 56,554 (number of actual non-defaulting clients)

- **Class 1 (Defaulting clients)**:
  - **Precision**: 0.17 (proportion of clients predicted to default that actually defaulted)
  - **Recall**: 0.68 (proportion of actual defaulting clients correctly predicted)
  - **F1-Score**: 0.28 (harmonic mean of precision and recall)
  - **Support**: 4,949 (number of actual defaulting clients)

- **Overall Metrics**:
  - **Accuracy**: 0.71 (proportion of total correct predictions)
  - **Macro Average**: Average precision, recall, and F1-score for both classes (treating all classes equally)
  - **Weighted Average**: Average precision, recall, and F1-score for both classes (considering the support of each class)

#### Observations and Insights

1. **Model Performance**:
   - The LightGBM model shows reasonable classification performance with an ROC AUC score of 0.698, indicating moderate discriminative ability between defaulting and non-defaulting clients.
   - The high precision for class 0 (0.96) indicates that the model is very good at predicting non-defaulting clients. However, the precision for class 1 (0.17) is low, meaning there are many false positives among the default predictions.

2. **Recall for Defaulting Clients (1.0)**:
   - The recall for class 1 (0.68) is fairly high, indicating that the model detects a significant proportion of defaulting clients. However, the low precision suggests that many of these positive predictions are incorrect.

3. **Class Imbalance**:
   - The model is still affected by class imbalance, as evidenced by the disparity in performance metrics between the two classes.
   - Using the `class_weight='balanced'` parameter helps improve the recall for the minority class (defaulting clients), but precision remains a challenge.

### Recommendations

1. **Improve Class Imbalance Handling**:
   - Continue experimenting with different resampling techniques such as SMOTE, under-sampling, or a combination of both.
   - Explore other parameters and configurations for LightGBM, such as adjustments to regularization and tree depth.

2. **Refine the Model**:
   - Conduct a more thorough hyperparameter search to further optimize the model.
   - Consider using other robust models for class imbalance, such as XGBoost or ensemble classifiers.

3. **Interpret the Results**:
   - Use model interpretation techniques such as SHAP to understand which features most influence the model’s predictions and potentially improve feature selection.

Focusing on these aspects can further improve the model's ability to better predict defaulting and non-defaulting clients.

# 4.Feature Importance

## 4.1 Global Feature Importance Using SHAP

The objective of this code is to:
1. **Initialize SHAP Explainer**: Create a SHAP explainer object for the best model's classifier using the training data.
2. **Compute SHAP Values**: Generate SHAP values for the test dataset to explain the model's predictions.
3. **Global Feature Importance**: Visualize the global feature importance using SHAP summary plot, which provides insights into how each feature contributes to the model's predictions.

In [10]:
import shap  # Import the SHAP library used for interpreting model predictions
import pandas as pd  # Import pandas for data manipulation

# Convert the data to DataFrame if it's not already one
X_train_df = pd.DataFrame(X_train)  # Convert the training data to DataFrame
X_val_df = pd.DataFrame(X_val_split)  # Convert the validation data to DataFrame
X_test_df = pd.DataFrame(X_test)  # Convert the test data to DataFrame

# Function to reduce memory usage by sampling data
def reduce_memory_usage(df, sample_fraction=0.1):
    """Reduces memory usage by sampling a fraction of the data."""
    return df.sample(frac=sample_fraction, random_state=42)

# Reduce memory usage by sampling
X_train_sampled = reduce_memory_usage(X_train_df)
X_val_sampled = reduce_memory_usage(X_val_df)
X_test_sampled = reduce_memory_usage(X_test_df)

# Check and convert categorical variables to numeric
X_train_sampled = pd.get_dummies(X_train_sampled)
X_val_sampled = pd.get_dummies(X_val_sampled)
X_test_sampled = pd.get_dummies(X_test_sampled)

# Align the training, validation, and testing data to have the same columns
X_train_sampled, X_val_sampled = X_train_sampled.align(X_val_sampled, join='inner', axis=1)
X_train_sampled, X_test_sampled = X_train_sampled.align(X_test_sampled, join='inner', axis=1)
X_val_sampled, X_test_sampled = X_val_sampled.align(X_test_sampled, join='inner', axis=1)

# Ensure the columns are in the same order
X_val_sampled = X_val_sampled[X_train_sampled.columns]
X_test_sampled = X_test_sampled[X_train_sampled.columns]

# Fit the explainer on the training data
explainer = shap.Explainer(best_model.named_steps['classifier'], X_train_sampled)
shap_values = explainer(X_test_sampled)  # Compute SHAP values for the test data

# Global feature importance
shap.summary_plot(shap_values, X_test_sampled)  # Visualize the global feature importance using SHAP summary plot

# Local explanation for a single instance
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0, :], X_test_sampled.iloc[0, :])


ValueError: shapes (247,) and (4108,) not aligned: 247 (dim 0) != 4108 (dim 0)

In [ ]:
# import shap  # Import the SHAP library used for interpreting model predictions
# import pandas as pd  # Import pandas for data manipulation
# 
# # Convert the data to a DataFrame if it's not already one
# X_train_split_df = pd.DataFrame(X_train_split)  # Convert the training data to a DataFrame
# X_val_split_df = pd.DataFrame(X_val_split)  # Convert the validation data to a DataFrame
# X_test_df = pd.DataFrame(X_test)  # Convert the test data to a DataFrame
# 
# # Function to reduce memory usage by sampling data
# def reduce_memory_usage(df, sample_fraction=0.1):  # Reduce memory usage by sampling to a fraction of the data
#     return df.sample(frac=sample_fraction, random_state=42)  # Sample a fraction of the data
# 
# # Reduce memory usage by sampling
# X_train_split_df_sampled = reduce_memory_usage(X_train_split_df)  # Sample the training data
# X_val_split_df_sampled = reduce_memory_usage(X_val_split_df)  # Sample the validation data
# X_test_df_sampled = reduce_memory_usage(X_test_df)  # Sample the test data
# 
# # Check and convert categorical variables to numeric
# X_train_split_df_sampled = pd.get_dummies(X_train_split_df_sampled)  # Convert categorical variables to numeric for training data
# X_val_split_df_sampled = pd.get_dummies(X_val_split_df_sampled)  # Convert categorical variables to numeric for validation data
# X_test_df_sampled = pd.get_dummies(X_test_df_sampled)  # Convert categorical variables to numeric for test data
# 
# # Align the training, validation, and testing data to have the same columns
# X_train_split_df_sampled, X_val_split_df_sampled = X_train_split_df_sampled.align(X_val_split_df_sampled, join='inner', axis=1)  # Align the training and validation data
# X_train_split_df_sampled, X_test_df_sampled = X_train_split_df_sampled.align(X_test_df_sampled, join='inner', axis=1)  # Align the training and test data
# X_val_split_df_sampled, X_test_df_sampled = X_val_split_df_sampled.align(X_test_df_sampled, join='inner', axis=1)  # Align the validation and test data
# 
# # Ensure the columns are in the same order
# X_val_split_df_sampled = X_val_split_df_sampled[X_train_split_df_sampled.columns]  # Ensure the columns are in the same order for validation data
# X_test_df_sampled = X_test_df_sampled[X_train_split_df_sampled.columns]  # Ensure the columns are in the same order for test data
# 
# # Fit the explainer on the training data
# explainer = shap.Explainer(best_model.named_steps['classifier'], X_train_split_df_sampled)  # Fit the SHAP explainer on the training data
# shap_values = explainer(X_val_split_df_sampled)  # Compute SHAP values for the validation data
# 
# # Global feature importance
# shap.summary_plot(shap_values, X_val_split_df_sampled)  # Visualize the global feature importance using SHAP summary plot
# 
# # Local explanation for a single instance
# shap.initjs()  # Initialize the SHAP JavaScript visualization
# shap.force_plot(explainer.expected_value, shap_values[0, :], X_val_split_df_sampled.iloc[0, :])  # Generate a SHAP force plot for a single instance







## 4.2 Local Feature Importance Using SHAP

The objective of this code is to:
1. **Initialize SHAP Visualization**: Set up SHAP's JavaScript visualization framework to enable interactive plots.
2. **Generate Local Explanation**: Create a SHAP force plot for a single instance from the test dataset, showing how each feature contributes to the model's prediction for that specific instance.
3. **Visualize Prediction Breakdown**: Provide a detailed breakdown of the prediction for the selected instance, illustrating the impact of each feature on the predicted value.

## Glossary

### Cross-Validation
A technique used to assess the performance of a model by splitting the dataset into multiple training and testing sets. This helps in understanding how the model will generalize to an independent dataset.

### GridSearchCV
A tool from `scikit-learn` that performs hyperparameter tuning by exhaustively searching through a specified parameter grid to find the best combination of hyperparameters for a given model, using cross-validation.

### Hyperparameter Tuning
The process of finding the optimal values for hyperparameters of a model, which are parameters that are not learned from data but set before the learning process begins. Examples include the number of trees in a random forest or the regularization strength in Ridge regression.

### ROC AUC Score
A performance measurement for classification problems at various threshold settings. ROC AUC represents the area under the Receiver Operating Characteristic curve, which plots the true positive rate against the false positive rate. A score closer to 1 indicates better performance.

### Confusion Matrix
A table used to describe the performance of a classification model by showing the actual vs. predicted classifications. It includes True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN).

### Classification Report
A detailed report showing the precision, recall, F1-score, and support for each class in a classification problem. This helps in understanding the performance of the model across different classes.

### Resampling Techniques
Methods used to adjust the class distribution of a dataset, commonly used to handle class imbalance. Examples include:
- **SMOTE (Synthetic Minority Over-sampling Technique)**: Generates synthetic samples for the minority class.
- **RandomUnderSampler**: Reduces the number of samples in the majority class.

### Pipeline
A tool from `scikit-learn` that allows for chaining multiple processing steps (e.g., data transformation and model fitting) into a single object. This ensures that all steps are applied consistently during both training and testing.

### SHAP (SHapley Additive exPlanations)
A method to explain individual predictions of machine learning models by assigning each feature an importance value. It helps in understanding how the model arrives at its predictions.

### SHAP Explainer
An object in the SHAP library that is used to calculate SHAP values for a given model. It helps in interpreting the contributions of each feature to the predictions.

### SHAP Values
Values calculated by the SHAP explainer that quantify the contribution of each feature to the model's prediction for a given instance. Higher absolute values indicate greater impact on the prediction.

### SHAP Summary Plot
A plot that visualizes the global importance of features by showing the distribution of SHAP values for each feature across all instances in the dataset. It helps in understanding which features are most influential for the model.

### SHAP Force Plot
A plot that provides a detailed breakdown of the contributions of each feature to a single instance's prediction, illustrating how different features push the prediction towards or away from the base value.

### Random Forest
An ensemble learning method that constructs multiple decision trees during training and outputs the mode of the classes (classification) or mean prediction (regression) of the individual trees.

### Ridge Classifier
A linear model for classification that includes L2 regularization (Ridge regression) to prevent overfitting by penalizing large coefficients.

### Class Weight
A parameter used to handle class imbalance by assigning different weights to different classes, typically to give more importance to the minority class. This can be set to 'balanced' to automatically adjust weights inversely proportional to class frequencies.

### RandomUnderSampler
A resampling technique that reduces the number of instances in the majority class by randomly sampling without replacement, used to balance the class distribution.

### SMOTE
A resampling technique that generates synthetic samples for the minority class by interpolating between existing minority class instances. It is used to balance the class distribution in the dataset.
